# 从静态到动态：循环神经网络（RNN）的序列建模革命

自从 MLP 开创了现代化神经网络后，序列数据（如文本、语音、时间序列）的处理一直面临着挑战。

感知器模型的输入只能是固定尺寸，如输入不定长的序列只能截断或填充，当然这样效果并不好。

而且 MLP 只会把整个输入当成一个整体，识别部分中的语义，为了解决 MLP 带来的局限，人们创造了循环神经网络。

RNN的诞生并非偶然，而是认知科学、神经科学、计算机理论共同推动的结果：

认知科学启发：

人类处理语言、记忆的方式具有时序性（如理解句子需逐词解析并保留上下文）。

神经科学基础：

大脑神经元间的反馈连接（如海马体的记忆循环）启发了循环结构的设计。

计算理论突破：

1982年John Hopfield提出Hopfield网络（能量模型），首次证明循环结构可存储模式。

RNN 的特点是每个时间步接收当前输入和上一步的隐藏状态，形成记忆流。下面是一个简单的循环神经网络实现

In [5]:
import torch

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 假设使用 32 个参数描述输入信息,64个参数描述输出信息（隐藏状态）
        # 那么循环神经网络的隐藏线性层输入参数是 32 + 64。因为循环神经网络的线性层需要循环
        self.i2h = torch.nn.Linear(32 + 64, 64)
        # 输出线性层的输出和模型输入一致，这样才能循环。
        self.h2o = torch.nn.Linear(64, 32)

    def forward(self, inputs: torch.Tensor, hidden: torch.Tensor):
        outputs = []
        for i in inputs:
            combined = torch.cat([i, hidden], dim=-1)
            hidden = torch.tanh(self.i2h(combined))
            o = self.h2o(hidden)
            outputs.append(o)
        return torch.stack(outputs, dim=0), hidden

    @torch.no_grad()
    def generate(self, inputs: torch.Tensor, timesize: int = 128):
        _, hidden = self(inputs, torch.zeros(64))
        outputs = []
        for _ in range(timesize):
            o = self.h2o(hidden)
            outputs.append(o)
            combined = torch.cat([o, hidden], dim=-1)
            hidden = torch.tanh(self.i2h(combined))

        return torch.stack(outputs, dim=0)

In [6]:
import random

model = RNN()
# 测试输入任意长度的张量，输出任意长度的张量。
inputs = torch.randn(random.randint(10, 20), 32)
print(inputs.shape)
outputs = model.generate(inputs, random.randint(20, 30))
print(outputs.shape)

torch.Size([19, 32])
torch.Size([21, 32])


RNN 同一组权重在时间步间复用，大幅减少参数量。

序列输出更具灵活性，支持一对多（如生成音乐）、多对一（如文本分类）、多对多（如机器翻译）。

每次隐藏状态的生成都依赖上一轮的隐藏状态，使得 RNN 能够利用前后文关联，进行时序依赖建模。

# RNN的致命缺陷：短期记忆问题

尽管 RNN 理论上可以记住任意长历史，但实际训练中只能学习短距离依赖，前序输入的隐藏状态就在权重连乘下指数衰减，约10个时间步内就会变得微乎其微。

而反向传播梯度需沿时间步连乘，前序输入几乎无法更新参数。

为了解决短期记忆问题，人们提出了很多基于RNN 的改进方案，下面我们会逐步实现一个十分经典的对 RNN 的改进：长短时记忆 LSTM

# 长短时记忆网络 LSTM

1990年代中期，尽管RNN在序列建模上迈出关键一步，但实际应用暴露了致命缺陷：

- 梯度消失：反向传播时，梯度随时间步指数衰减（如0.9^100 ≈ 2.6e-5），导致早期时间步的参数无法更新。

- 无关信息堆积：隐藏状态被动累积所有历史信息，无法区分重要/次要内容（如预测“I grew up in France… I speak fluent _”时，模型可能遗忘关键词“France”）。

1997年，Sepp Hochreiter与Jürgen Schmidhuber提出LSTM（Long Short-Term Memory），首次通过门控机制实现记忆的精准控制。


**LSTM的核心机制：三重门控与细胞状态**

|组件|功能|数学表达|
|----|----|----|
|遗忘门（Forget Gate）|决定丢弃哪些历史信息|f_t = σ(W_f·[h_{t-1}, x_t] + b_f)|
|输入门（Input Gate）|筛选当前输入的有用部分|i_t = σ(W_i·[h_{t-1}, x_t] + b_i)|
|输出门（Output Gate）|控制当前状态的输出强度|o_t = σ(W_o·[h_{t-1}, x_t] + b_o)|
|细胞状态（Cell State）|长期记忆的“高速公路”|C_t = f_t⊙C_{t-1} + i_t⊙tanh(W_C·[h_{t-1}, x_t] + b_C)|
|隐藏状态（Hidden State）|短期记忆输出|h_t = o_t⊙tanh(C_t)|


对抗梯度消失：细胞状态的更新路径是直接相加（C_t = f_t⊙C_{t-1} + ...）允许梯度无损流动（若遗忘门≈1）。

选择性记忆：输入门和遗忘门动态决定保留/丢弃信息，记住关键上下文。

In [7]:
class LSTM(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # 假设使用 32 个参数描述输入信息,64个参数描述输出信息（隐藏状态）
        input_size = 32
        hidden_size = 64
        combined_size = hidden_size + input_size
        # 遗忘门
        self.forget_gate = torch.nn.Linear(combined_size, hidden_size)
        # 输入门
        self.in_gate = torch.nn.Linear(combined_size, hidden_size)
        # 备选细胞状态
        self.candidate_cell_state = torch.nn.Linear(combined_size, hidden_size)
        # 输出门
        self.out_gate = torch.nn.Linear(combined_size, hidden_size)

        self.h2o = torch.nn.Linear(hidden_size, input_size)

    def forward(self, inputs: torch.Tensor, hidden: torch.Tensor, cellstates: torch.Tensor):
        outputs = []
        for i in inputs:
            combined = torch.cat((i, hidden), dim=-1)
            # combined 是输入和隐藏状态拼接，形状为 [input_size + hidden_size]
            forgetgate = torch.sigmoid(self.forget_gate(combined))
            # 输入门为了过滤输入噪声，只有重要的信息才会进入细胞状态
            ingate = torch.sigmoid(self.in_gate(combined))
            # 通过张量逐元素乘法使得（上个细胞状态经过遗忘 + 新的细胞状态经过输入选择）合成为当前细胞状态
            cellstates = (cellstates * forgetgate) + (torch.tanh(self.candidate_cell_state(combined)) * ingate)
            # 输出门过滤输出噪声，只有重要的信息才会进入隐藏状态
            outgate = torch.sigmoid(self.out_gate(combined))
            hidden = torch.tanh(cellstates) * outgate
            o = self.h2o(hidden)
            outputs.append(o)
        return torch.stack(outputs, dim=0), hidden, cellstates

    @torch.no_grad()
    def generate(self, inputs: torch.Tensor, timesize: int = 128):
        _, hidden, cellstates = self(inputs, torch.zeros(64), torch.zeros(64))
        outputs = []
        for _ in range(timesize):
            o = self.h2o(hidden)
            outputs.append(o)
            combined = torch.cat([o, hidden], dim=-1)
            forgetgate = torch.sigmoid(self.forget_gate(combined))
            ingate = torch.sigmoid(self.in_gate(combined))
            cellstates = (cellstates * forgetgate) + (torch.tanh(self.candidate_cell_state(combined)) * ingate)
            outgate = torch.sigmoid(self.out_gate(combined))
            hidden = torch.tanh(cellstates) * outgate
        return torch.stack(outputs, dim=0)

In [8]:
model = LSTM()
# 测试输入任意长度的张量，输出任意长度的张量。
inputs = torch.randn(random.randint(10, 20), 32)
print(inputs.shape)
outputs = model.generate(inputs, random.randint(20, 30))
print(outputs.shape)

torch.Size([10, 32])
torch.Size([21, 32])
